<a href="https://colab.research.google.com/github/fowardelcac/Titanic_2.0/blob/main/Modelos/Modelado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/fowardelcac/Titanic_2.0.git

Cloning into 'Titanic_2.0'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 61 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (61/61), 104.57 KiB | 1.14 MiB/s, done.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/content/Titanic_2.0/Datasets_originales/Procesamiento/train_procesado.csv').set_index('PassengerId')

In [ ]:
df.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [ ]:
df[df.Survived == 0]

,Survived,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,,
1,0,3,0,22.0,1,0
5,0,3,0,35.0,0,0
6,0,3,0,24.0,0,0
7,0,1,0,54.0,0,0
8,0,3,0,2.0,3,1
...,...,...,...,...,...,...
885,0,3,0,25.0,0,0
886,0,3,1,39.0,0,5
887,0,2,0,27.0,0,0


In [ ]:
round((len(df[df.Survived == 0]) / len(df.Survived) )* 100, 2)

61.62

In [ ]:
X = df[['Pclass', 'Sex', 'Age']].values.reshape((-1, 3))
y = df.Survived

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=42)

In [ ]:
round((len(y_train[y_train == 0]) / len(y_train)) * 100, 2)

61.65730337078652

In [ ]:
round((len(y_test[y_test == 0]) / len(y_test)) * 100, 2)

61.45

In [ ]:
round((len(X_train[X_train[:, 0] == 1]) / len(X_train[:, 0])) * 100, 2)

24.02

In [ ]:
round((len(df[df.Pclass == 1] )/ len(df.Pclass)) * 100, 2)

24.24

In [ ]:
dic_rdos = {}

In [ ]:
def modelado(modelo, X_train, X_test, y_train, y_test ):
  model = modelo().fit(X_train, y_train)
  y_pred = model.predict(X_test)
  return accuracy_score(y_test, y_pred)


# Regresion logistica

In [ ]:
dic_rdos['RL'] =  modelado(LogisticRegression,  X_train, X_test, y_train, y_test)

# Arbol

In [ ]:
dic_rdos['Arbol'] =  modelado(DecisionTreeClassifier,  X_train, X_test, y_train, y_test)

# Random forest

In [ ]:
dic_rdos['RF'] =  modelado(RandomForestClassifier,  X_train, X_test, y_train, y_test)

# KNN

In [ ]:
dic_rdos['KN'] =  modelado(KNeighborsClassifier,  X_train, X_test, y_train, y_test)

In [ ]:
dic_rdos

{'RL': 0.7653631284916201,
 'Arbol': 0.7877094972067039,
 'RF': 0.776536312849162,
 'KN': 0.7094972067039106}

# Grid arbol

In [ ]:
estimador = DecisionTreeClassifier()
estimador.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 5, 10],
    'max_features': ['sqrt'],
    'min_samples_split': [4, 5, 10]
    }


In [ ]:
grid = GridSearchCV(estimador, params, cv = 5)
grid.fit(X_train, y_train)

resultados = grid.cv_results_
mean_scores = resultados['mean_test_score']
mejor_indice = np.argmax(mean_scores)
mejores_parametros = resultados['params'][mejor_indice]
print("Mejores hiperparámetros:", mejores_parametros)
print("Accuracy score:", mean_scores[mejor_indice])

Mejores hiperparámetros: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 4}
Accuracy score: 0.796395154141633


In [ ]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_split': 4}

In [ ]:
resultados['params'][10]

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 5}

In [ ]:
estimador = DecisionTreeClassifier(criterion = 'gini', max_depth= 5, max_features= 'sqrt', min_samples_split = 10).fit(X_train, y_train)
y_pred = estimador.predict(X_test)
accuracy_score(y_test, y_pred)

0.776536312849162

# Grid RF

In [ ]:
estimador_RF = RandomForestClassifier()

In [ ]:
params = {
    'n_estimators': [100, 150, 200, 250, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 5]
    }

grid = GridSearchCV(estimador_RF, params, cv = 5)
grid.fit(X_train, y_train)

resultados = grid.cv_results_
mean_scores = resultados['mean_test_score']
mejor_indice = np.argmax(mean_scores)
mejores_parametros = resultados['params'][mejor_indice]
print("Mejores hiperparámetros:", mejores_parametros)
print("Accuracy score:", mean_scores[mejor_indice])


Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 200}
Accuracy score: 0.8034177090515119


In [ ]:
rf_ajust = RandomForestClassifier(criterion = 'entropy', max_depth = 2, n_estimators = 250).fit(X_train, y_train)
y_pred = rf_ajust.predict(X_test)
ac = accuracy_score(y_test, y_pred)
print(ac)

0.7821229050279329


# grid RL

In [ ]:
estimador_RL = LogisticRegression()
parametros = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['liblinear']
}

grid = GridSearchCV(estimador_RL, parametros, cv = 5)
grid.fit(X_train, y_train)
resultados = grid.cv_results_
mean_scores = resultados['mean_test_score']
mejor_indice = np.argmax(mean_scores)
mejores_parametros = resultados['params'][mejor_indice]
print("Mejores hiperparámetros:", mejores_parametros)
print("Accuracy score:", mean_scores[mejor_indice])

Mejores hiperparámetros: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy score: 0.7992120555500838


In [ ]:
rl_ajust = LogisticRegression(C = 0.1, penalty = 'l2', solver = 'liblinear').fit(X_train, y_train)
y_pred = rl_ajust.predict(X_test)
ac = accuracy_score(y_test, y_pred)
print(ac)

0.7653631284916201


In [ ]:
 =